# Методы сбора и обработки данных из сети Интернет
___
___
## Урок 2. Парсинг данных. HTML, DOM, XPath
___

### Задачи

1. Написать приложение и функцию, которые собирают основные новости с сайта на выбор dzen.ru, lenta.ru, mail.ru . Для парсинга использовать XPath
Структура данных должна содержать:
* название источника
* наименование новости
* ссылку на новость
* дата публикации
минимум один сайт максимум все

In [1]:
from pprint import pprint
from lxml import html
import requests
import pandas as pd

In [2]:
USER_AGENT = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 13_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'
URL = 'https://lenta.ru/'

In [3]:
headers = {
    'User-Agent': USER_AGENT,
}

params = {
    'sso_failed': '',
}

In [4]:
def getContent(url, headers=None, params=None):
    response = requests.get(url, headers=headers, params=params)
    content_dom = html.fromstring(response.text)
    print(response.url)
    return content_dom

In [5]:
def parser(content_dom):
    news_container = content_dom.xpath("//a[contains(@class, 'card-')]")
    news = []
    for new_container in news_container:
        new_source = new_container.xpath(".//*[name()='svg']/*[name()='use']/attribute::*")
        if len(new_source) == 1:
            new_source = new_source[0].split('ui-label_')
            if len(new_source) == 2:
                new_source = new_source[1]
            else:
                new_source = 'Lenta.ru'
        else:
            new_source = 'Lenta.ru' 
        new_text = new_container.xpath(".//span[contains(@class, 'card-')]/text()")
        if len(new_text) == 1:
            new_text = new_text[0]
        else:
            new_text = None  
        new_link = URL + new_container.xpath("./@href")[0]
        
        new_date = new_container.xpath(".//time/text()")
        if len(new_date) == 1:
            new_date = new_date[0]
        else:
            new_date = None
        news_dict = {
            'new_source': new_source,
            'new_text': new_text,
            'new_link': new_link,
            'new_date': new_date,
        }
        news.append(news_dict)
        
    return news

In [6]:
dom, response = getContent(URL, headers=headers)

https://lenta.ru/


In [7]:
pd.DataFrame(parser(dom))

,new_source,new_text,new_link,new_date
0,Lenta.ru,None,https://lenta.ru//news/2023/03/03/soutnikv/,17:24
1,Lenta.ru,Стало известно о новом мегапроекте в России,https://lenta.ru//extlink/2023/03/03/mega/?int...,17:47
2,Lenta.ru,76-летняя певица Шер сменила имидж,https://lenta.ru//news/2023/03/03/cher_blondi/,17:46
3,Lenta.ru,Врач назвала способы восстановиться после прие...,https://lenta.ru//news/2023/03/03/microflora/,17:43
4,Lenta.ru,В Москве секс-работник убил мужчину во время с...,https://lenta.ru/https://moslenta.ru/news/city...,17:36
...,...,...,...,...
138,Lenta.ru,Как они изменят жизнь студентов и регионов?,https://lenta.ru//articles/2023/02/28/studgrd/,"17:05, 28 февраля 2023"
139,Lenta.ru,Минобороны отчиталось о сбитом украинском Су-2...,https://lenta.ru//news/2023/03/03/brieff/,15:11
140,Lenta.ru,"Диссидент, идеолог Кремля, «дурачок для битья...",https://lenta.ru//articles/2023/02/27/glebpavl...,"18:43, 27 февраля 2023"
141,Lenta.ru,Российские военные уничтожили более 200 бойцов...,https://lenta.ru//news/2023/03/03/krasnolim/,15:10
